In [109]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import os
import gzip
import numpy as np
import json
import ast
import geopandas
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon, MultiPolygon
from haversine import haversine, Unit
from shapely import wkt
from sklearn.metrics.pairwise import cosine_similarity
import math

year = 2020

In [2]:
answers = {'cbg': [], 'safegraph_place_id': []}

# A-D

In [3]:
table_2018 = pd.read_csv('data/2_data_for_PSO_calibration/table_2018.csv')
cbgs = list(table_2018['A_cbg'].unique())
cbgs = [str(c) for c in cbgs]

In [4]:
def get_stores_foot(walk_path_filtered):
    datapath_all_filtered = os.listdir(walk_path_filtered)
    datapath_all_filtered = [walk_path_filtered +'/'+i for i in datapath_all_filtered if '.DS_Store' not in i if '.ipynb_checkpoints' not in i]
    todo_list = [i for i in datapath_all_filtered if str(year) in i]
    data_filtered = [pd.read_csv(i).rename(columns = {'Unnamed: 0': 'cbg'}).set_index('cbg') for i in todo_list]
    final_data = sum(data_filtered).reset_index()
    return final_data

In [5]:
visitors = get_stores_foot('weekly_foot_traffic_matrix_nyc_2020') # We do not upload this data because the information is raw from SafeGraph
visitors

,cbg,sg:94b69491e8394408a41b4eaf79700a9a,sg:dccb7bd06b53430e9aa215f4e47623f8,sg:40c47fece0ba4897b01c7e28efb2731a,sg:f51c84ff03e942c79e0fbf6e77caf930,sg:7bf321e231b74c2f8e5a83727daca049,sg:b261ab19c88f4f68bb8efd01f0bea507,sg:3f8d237767dc4dda8c8ed80f0e7d359d,sg:1b84da99a0a545c99110a376a840293c,sg:dcef1702bffa47df9ff2ea7a2373bdbb,...,sg:b97dc59beab442a1833e4f2dbaf208f5,sg:0156d4380d684d8dae668182ad77336d,sg:098221490c474e6cbc4ca8a2162f7170,sg:8d376e7211074b6f8da481353e0a4627,sg:543c0a12cee5429cbd6418a51cffa2fa,sg:4217cdcdfa1a4bf7995658ea27daa03e,sg:78948e77d30e426d90d4ae1998f336c0,sg:eb3b350749604f60abc470b0ae0b4822,sg:15682cf1eadb423297c1e634d13c0377,sg:21870f3478e947c09c4e556b14b903c5
0,360810140001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,360810141001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,360810142012,4,0,8,4,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,0
3,360810142011,0,4,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,360810142021,0,0,4,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6488,360470271002,0,0,0,4,4,0,16,0,0,...,0,0,0,0,0,0,0,0,0,0
6489,360470280002,0,0,0,0,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6490,360470282002,0,0,0,0,31,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6491,360470282003,0,0,0,0,28,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
ids = visitors.columns[1:].tolist()

In [7]:
visitors

,cbg,sg:94b69491e8394408a41b4eaf79700a9a,sg:dccb7bd06b53430e9aa215f4e47623f8,sg:40c47fece0ba4897b01c7e28efb2731a,sg:f51c84ff03e942c79e0fbf6e77caf930,sg:7bf321e231b74c2f8e5a83727daca049,sg:b261ab19c88f4f68bb8efd01f0bea507,sg:3f8d237767dc4dda8c8ed80f0e7d359d,sg:1b84da99a0a545c99110a376a840293c,sg:dcef1702bffa47df9ff2ea7a2373bdbb,...,sg:b97dc59beab442a1833e4f2dbaf208f5,sg:0156d4380d684d8dae668182ad77336d,sg:098221490c474e6cbc4ca8a2162f7170,sg:8d376e7211074b6f8da481353e0a4627,sg:543c0a12cee5429cbd6418a51cffa2fa,sg:4217cdcdfa1a4bf7995658ea27daa03e,sg:78948e77d30e426d90d4ae1998f336c0,sg:eb3b350749604f60abc470b0ae0b4822,sg:15682cf1eadb423297c1e634d13c0377,sg:21870f3478e947c09c4e556b14b903c5
0,360810140001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,360810141001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,360810142012,4,0,8,4,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,0
3,360810142011,0,4,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,360810142021,0,0,4,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6488,360470271002,0,0,0,4,4,0,16,0,0,...,0,0,0,0,0,0,0,0,0,0
6489,360470280002,0,0,0,0,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6490,360470282002,0,0,0,0,31,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6491,360470282003,0,0,0,0,28,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
visitors = visitors.set_index('cbg')
visitors_sum = visitors.sum(axis = 1)
visitors_percentage = visitors.div(visitors_sum, axis='rows')
visitors = visitors.reset_index()
visitors_percentage = visitors_percentage.reset_index()

In [9]:
cbgs = visitors.cbg.tolist()

In [10]:
def turn_to_dict_visitors(df):
    dict_temp = {}
    for i,cont in df.iterrows():
        dict_temp[cont.cbg] = {}
        for store_id in ids:
            dict_temp[cont.cbg][store_id] = cont[store_id]
    return dict_temp

In [11]:
visitors_dict = turn_to_dict_visitors(visitors)

In [12]:
for c in cbgs:
    for i in ids:
        answers['cbg'].append(c)
        answers['safegraph_place_id'].append(i)
answers_df = pd.DataFrame(answers)

In [13]:
answers_df

,cbg,safegraph_place_id
0,360810140001,sg:94b69491e8394408a41b4eaf79700a9a
1,360810140001,sg:dccb7bd06b53430e9aa215f4e47623f8
2,360810140001,sg:40c47fece0ba4897b01c7e28efb2731a
3,360810140001,sg:f51c84ff03e942c79e0fbf6e77caf930
4,360810140001,sg:7bf321e231b74c2f8e5a83727daca049
...,...,...
1831021,360470286004,sg:4217cdcdfa1a4bf7995658ea27daa03e
1831022,360470286004,sg:78948e77d30e426d90d4ae1998f336c0
1831023,360470286004,sg:eb3b350749604f60abc470b0ae0b4822
1831024,360470286004,sg:15682cf1eadb423297c1e634d13c0377


In [14]:
def get_visitors_data(dict_here, row):
    return dict_here[row.cbg][row.safegraph_place_id]

answers_df['Number_of_Visits_'+str(year)] = answers_df.apply(lambda row: get_visitors_data(visitors_dict, row), axis=1)

In [15]:
answers_df

,cbg,safegraph_place_id,Number_of_Visits_2020
0,360810140001,sg:94b69491e8394408a41b4eaf79700a9a,0
1,360810140001,sg:dccb7bd06b53430e9aa215f4e47623f8,0
2,360810140001,sg:40c47fece0ba4897b01c7e28efb2731a,0
3,360810140001,sg:f51c84ff03e942c79e0fbf6e77caf930,0
4,360810140001,sg:7bf321e231b74c2f8e5a83727daca049,0
...,...,...,...
1831021,360470286004,sg:4217cdcdfa1a4bf7995658ea27daa03e,0
1831022,360470286004,sg:78948e77d30e426d90d4ae1998f336c0,0
1831023,360470286004,sg:eb3b350749604f60abc470b0ae0b4822,0
1831024,360470286004,sg:15682cf1eadb423297c1e634d13c0377,0


# E and N

In [16]:
with open('nyc_cbgs.json') as f:
    cbg_data = json.load(f)
cbg_info_full = cbg_data['features']

In [17]:
shp_path = 'data/1_data_for_extracting_information/tl_2017_36_bg/tl_2017_36_bg.shp'
shp_df = geopandas.GeoDataFrame.from_file(shp_path,encoding = 'gb18030')
cbg_info = {}
cbg_id = shp_df['GEOID'].tolist()
cbg_geo = shp_df['geometry'].tolist()
for i in range(len(cbg_id)):
    if int(cbg_id[i]) in cbgs:
        cbg_info[cbg_id[i]] = cbg_geo[i]

In [18]:
for i in cbgs:
    if str(i) not in cbg_id:
        print(i)

In [19]:
len(cbg_info)

6493

In [22]:
def get_census_data(path, col):
    temp = pd.read_csv(path)
    temp['census_block_group'] = temp['census_block_group'].apply(lambda x: int(x))
    temp = temp[temp['census_block_group'].isin(cbgs)]    
    cols = ['census_block_group']
    cols.extend(col)
    temp = temp[cols]
    temp = temp.fillna(0)
    temp['cbg_geometry'] = temp['census_block_group'].apply(lambda x: cbg_info[str(x)])
    temp = GeoDataFrame(temp, geometry = 'cbg_geometry')    
    return temp

In [23]:
income = 'data/1_data_for_extracting_information/cbg_b19.csv'
population = 'data/1_data_for_extracting_information/cbg_b25.csv'
income_df = get_census_data(income,['B19049e1'])
pop_df = get_census_data(population, ['B25033e1'])
census_nyc = pd.merge(pop_df, income_df.drop(columns = ['cbg_geometry']), on = 'census_block_group').rename(
    columns={"B25033e1": "population", "B19049e1": "median_income"})

In [24]:
census_gdf = geopandas.GeoDataFrame(census_nyc, geometry = 'cbg_geometry')

In [25]:
cbg_pop_dict = {}
cbg_mincome_dict = {}
for i,cont in census_gdf.iterrows():
    cbg_pop_dict[cont.census_block_group] = cont.population
    cbg_mincome_dict[cont.census_block_group] = cont.median_income

In [26]:
answers_df['Population_in_this_cbg'] = answers_df['cbg'].apply(lambda x: cbg_pop_dict[x])
answers_df['Median_Income_in_this_cbg'] = answers_df['cbg'].apply(lambda x: cbg_mincome_dict[x])

In [27]:
answers_df

,cbg,safegraph_place_id,Number_of_Visits_2020,Population_in_this_cbg,Median_Income_in_this_cbg
0,360810140001,sg:94b69491e8394408a41b4eaf79700a9a,0,2187,69750.0
1,360810140001,sg:dccb7bd06b53430e9aa215f4e47623f8,0,2187,69750.0
2,360810140001,sg:40c47fece0ba4897b01c7e28efb2731a,0,2187,69750.0
3,360810140001,sg:f51c84ff03e942c79e0fbf6e77caf930,0,2187,69750.0
4,360810140001,sg:7bf321e231b74c2f8e5a83727daca049,0,2187,69750.0
...,...,...,...,...,...
1831021,360470286004,sg:4217cdcdfa1a4bf7995658ea27daa03e,0,1147,59306.0
1831022,360470286004,sg:78948e77d30e426d90d4ae1998f336c0,0,1147,59306.0
1831023,360470286004,sg:eb3b350749604f60abc470b0ae0b4822,0,1147,59306.0
1831024,360470286004,sg:15682cf1eadb423297c1e634d13c0377,0,1147,59306.0


# F and H

In [28]:
poi_nyc = pd.read_csv('data/1_data_for_extracting_information/poi_NYC.csv')
square_data = pd.read_csv('data/1_data_for_extracting_information/SafeGraphPlacesGeoSupplementSquareFeet_202008.csv')

In [29]:
stores_with_poi_data = poi_nyc[poi_nyc['safegraph_place_id'].isin(visitors.columns.tolist())][['safegraph_place_id', 'longitude', 'latitude']]
square_data = square_data[square_data['safegraph_place_id'].isin(visitors.columns.tolist())][['safegraph_place_id', 'area_square_feet']]
geo_area_stores = pd.merge(stores_with_poi_data, square_data, on = 'safegraph_place_id')

In [30]:
geo_area_stores = geo_area_stores.append({'safegraph_place_id':'aggregated_517E','longitude': -73.93100799999999, 'latitude': 40.795408,
                                          'area_square_feet':169356}, ignore_index=True)
geo_area_stores = geo_area_stores.append({'safegraph_place_id':'aggregated_4024_College','longitude': -73.834744, 'latitude': 40.757749,
                                          'area_square_feet':214735}, ignore_index=True)
geo_area_stores = geo_area_stores.append({'safegraph_place_id':'aggregated_40W','longitude': -73.907558, 'latitude': 40.872776, 
                                          'area_square_feet':183593}, ignore_index=True)

In [31]:
geo_area_stores

,safegraph_place_id,longitude,latitude,area_square_feet
0,sg:1b84da99a0a545c99110a376a840293c,-74.191397,40.600690,22192
1,sg:6eb37e997f3e4ea68c4ffda26c95ebf2,-73.811354,40.587022,11080
2,sg:670f3d81cc5f44f5835954f0285f57af,-74.073002,40.617067,4740
3,sg:a1eb0e6f38994e40859b4bf5e9fc15d4,-74.156726,40.624749,13175
4,sg:0d9fe727bf634df3b1f5c9f7460f75e4,-73.916358,40.806030,18404
...,...,...,...,...
277,sg:84a8615ac25b455bb1d01ec4c93cdd1f,-74.165371,40.583277,96668
278,sg:ede0c3b8b3a143848181ee2430390072,-73.908321,40.659248,5885
279,aggregated_517E,-73.931008,40.795408,169356
280,aggregated_4024_College,-73.834744,40.757749,214735


In [32]:
def get_cbg(p1):
    for k,v in cbg_info.items():
        if p1.within(v):
            return k
    return 'No'

In [33]:
geometry_poi = [Point(xy) for xy in zip(geo_area_stores.longitude, geo_area_stores.latitude)]
geo_area_stores_gdf = GeoDataFrame(geo_area_stores, crs="EPSG:4326", geometry=geometry_poi)
geo_area_stores_gdf['geo'] = geo_area_stores_gdf['geometry'].apply(lambda x: (x.y, x.x))
geo_area_stores_gdf['cbg'] = geo_area_stores['geometry'].apply(lambda x: get_cbg(x))
geo_area_stores_gdf = geo_area_stores_gdf[geo_area_stores_gdf['cbg'] != 'No']
geo_area_stores_gdf['cbg'] = geo_area_stores_gdf['cbg'].astype('int64')

In [34]:
final_stores = geo_area_stores_gdf['safegraph_place_id'].tolist()

In [35]:
geo_area_stores_gdf['cbg_population'] = geo_area_stores_gdf['cbg'].apply(lambda x: cbg_pop_dict[x])
geo_area_stores_gdf['cbg_median_income'] = geo_area_stores_gdf['cbg'].apply(lambda x: cbg_mincome_dict[int(x)])

In [36]:
store_pop_dict = {}
store_mincome_dict = {}
store_area_dict = {}
for i, cont in geo_area_stores_gdf.iterrows():
    store_pop_dict[cont.safegraph_place_id] = cont.cbg_population
    store_mincome_dict[cont.safegraph_place_id] = cont.cbg_median_income
    store_area_dict[cont.safegraph_place_id] = cont.area_square_feet

In [37]:
geo_area_stores_gdf.head()

,safegraph_place_id,longitude,latitude,area_square_feet,geometry,geo,cbg,cbg_population,cbg_median_income
0,sg:1b84da99a0a545c99110a376a840293c,-74.191397,40.600690,22192,POINT (-74.19140 40.60069),"(40.60069, -74.191397)",360850291021,922,76250.0
1,sg:6eb37e997f3e4ea68c4ffda26c95ebf2,-73.811354,40.587022,11080,POINT (-73.81135 40.58702),"(40.587022, -73.81135400000002)",360810942032,1040,50667.0
2,sg:670f3d81cc5f44f5835954f0285f57af,-74.073002,40.617067,4740,POINT (-74.07300 40.61707),"(40.617067, -74.073002)",360850008001,1407,68259.0
3,sg:a1eb0e6f38994e40859b4bf5e9fc15d4,-74.156726,40.624749,13175,POINT (-74.15673 40.62475),"(40.624749, -74.156726)",360850303024,409,102656.0
4,sg:0d9fe727bf634df3b1f5c9f7460f75e4,-73.916358,40.806030,18404,POINT (-73.91636 40.80603),"(40.80603, -73.916358)",360050027012,1670,23066.0


In [38]:
answers_df = answers_df[answers_df['safegraph_place_id'].isin(final_stores)]

In [39]:
answers_df['Population_where_store_is'] = answers_df['safegraph_place_id'].apply(lambda x: store_pop_dict[x])
answers_df['Median_Income_where_store_is'] = answers_df['safegraph_place_id'].apply(lambda x: store_mincome_dict[x])

In [40]:
answers_df['Area_of_store'] = answers_df['safegraph_place_id'].apply(lambda x: store_area_dict[x])
answers_df

,cbg,safegraph_place_id,Number_of_Visits_2020,Population_in_this_cbg,Median_Income_in_this_cbg,Population_where_store_is,Median_Income_where_store_is,Area_of_store
0,360810140001,sg:94b69491e8394408a41b4eaf79700a9a,0,2187,69750.0,1682,54861.0,12179
1,360810140001,sg:dccb7bd06b53430e9aa215f4e47623f8,0,2187,69750.0,2453,41298.0,3715
2,360810140001,sg:40c47fece0ba4897b01c7e28efb2731a,0,2187,69750.0,1407,76288.0,472341
3,360810140001,sg:f51c84ff03e942c79e0fbf6e77caf930,0,2187,69750.0,1372,11818.0,4867
4,360810140001,sg:7bf321e231b74c2f8e5a83727daca049,0,2187,69750.0,1639,50809.0,137041
...,...,...,...,...,...,...,...,...
1831021,360470286004,sg:4217cdcdfa1a4bf7995658ea27daa03e,0,1147,59306.0,1058,40822.0,2370
1831022,360470286004,sg:78948e77d30e426d90d4ae1998f336c0,0,1147,59306.0,142,0.0,133161
1831023,360470286004,sg:eb3b350749604f60abc470b0ae0b4822,0,1147,59306.0,1491,16385.0,1626
1831024,360470286004,sg:15682cf1eadb423297c1e634d13c0377,0,1147,59306.0,1283,152083.0,84616


# G

In [41]:
def get_geo(cbg):
    centroid_info = cbg_info[str(cbg)].centroid
    return float(centroid_info.y), float(centroid_info.x)

In [42]:
store_geo_dict = {}
for i,cont in geo_area_stores_gdf.iterrows():
    store_geo_dict[cont.safegraph_place_id] = {}
    store_geo_dict[cont.safegraph_place_id]['geometry'] = cont.geometry
    store_geo_dict[cont.safegraph_place_id]['geo'] = cont.geo

In [43]:
def get_haversine(row):
    geo_cbg = get_geo(int(row.cbg))
    store_location = store_geo_dict[row.safegraph_place_id]['geo']
    return haversine(store_location, geo_cbg, unit = Unit.MILES)

In [44]:
answers_df['Distance_between_cbg_and_store'] = answers_df.apply(lambda row: get_haversine(row), axis=1)

In [45]:
answers_df

,cbg,safegraph_place_id,Number_of_Visits_2020,Population_in_this_cbg,Median_Income_in_this_cbg,Population_where_store_is,Median_Income_where_store_is,Area_of_store,Distance_between_cbg_and_store
0,360810140001,sg:94b69491e8394408a41b4eaf79700a9a,0,2187,69750.0,1682,54861.0,12179,8.962305
1,360810140001,sg:dccb7bd06b53430e9aa215f4e47623f8,0,2187,69750.0,2453,41298.0,3715,8.322993
2,360810140001,sg:40c47fece0ba4897b01c7e28efb2731a,0,2187,69750.0,1407,76288.0,472341,3.419066
3,360810140001,sg:f51c84ff03e942c79e0fbf6e77caf930,0,2187,69750.0,1372,11818.0,4867,9.443017
4,360810140001,sg:7bf321e231b74c2f8e5a83727daca049,0,2187,69750.0,1639,50809.0,137041,11.710228
...,...,...,...,...,...,...,...,...,...
1831021,360470286004,sg:4217cdcdfa1a4bf7995658ea27daa03e,0,1147,59306.0,1058,40822.0,2370,6.062409
1831022,360470286004,sg:78948e77d30e426d90d4ae1998f336c0,0,1147,59306.0,142,0.0,133161,10.430985
1831023,360470286004,sg:eb3b350749604f60abc470b0ae0b4822,0,1147,59306.0,1491,16385.0,1626,18.916934
1831024,360470286004,sg:15682cf1eadb423297c1e634d13c0377,0,1147,59306.0,1283,152083.0,84616,11.337666


# I

In [46]:
parking_1 = pd.read_csv('data/1_data_for_extracting_information/core_poi-geometry.csv')
parking_2 = pd.read_csv('data/1_data_for_extracting_information/first_step_data_with_competitors.csv')

In [47]:
parking_dict = {}
for i,cont in parking_1.iterrows():
    if cont.safegraph_place_id in final_stores:
        if cont.includes_parking_lot == 'False':
            parking_dict[cont.safegraph_place_id] = 0
        else:
            parking_dict[cont.safegraph_place_id] = 1
for i,cont in parking_2.iterrows():
    if cont.safegraph_place_id in final_stores:
        parking_dict[cont.safegraph_place_id] = cont.includes_parking_lot

for s in final_stores:
    if s not in parking_dict.keys():
        parking_dict[s] = 0.0001

In [48]:
answers_df['Includes_parking_lot'] = answers_df['safegraph_place_id'].apply(lambda x: parking_dict[x])

In [49]:
answers_df

,cbg,safegraph_place_id,Number_of_Visits_2020,Population_in_this_cbg,Median_Income_in_this_cbg,Population_where_store_is,Median_Income_where_store_is,Area_of_store,Distance_between_cbg_and_store,Includes_parking_lot
0,360810140001,sg:94b69491e8394408a41b4eaf79700a9a,0,2187,69750.0,1682,54861.0,12179,8.962305,0.0001
1,360810140001,sg:dccb7bd06b53430e9aa215f4e47623f8,0,2187,69750.0,2453,41298.0,3715,8.322993,1.0000
2,360810140001,sg:40c47fece0ba4897b01c7e28efb2731a,0,2187,69750.0,1407,76288.0,472341,3.419066,0.0001
3,360810140001,sg:f51c84ff03e942c79e0fbf6e77caf930,0,2187,69750.0,1372,11818.0,4867,9.443017,0.0001
4,360810140001,sg:7bf321e231b74c2f8e5a83727daca049,0,2187,69750.0,1639,50809.0,137041,11.710228,0.0001
...,...,...,...,...,...,...,...,...,...,...
1831021,360470286004,sg:4217cdcdfa1a4bf7995658ea27daa03e,0,1147,59306.0,1058,40822.0,2370,6.062409,0.0001
1831022,360470286004,sg:78948e77d30e426d90d4ae1998f336c0,0,1147,59306.0,142,0.0,133161,10.430985,0.0000
1831023,360470286004,sg:eb3b350749604f60abc470b0ae0b4822,0,1147,59306.0,1491,16385.0,1626,18.916934,0.0000
1831024,360470286004,sg:15682cf1eadb423297c1e634d13c0377,0,1147,59306.0,1283,152083.0,84616,11.337666,0.0001


# K and L

In [50]:
poi_diversity = pd.read_csv('data/1_data_for_extracting_information/poi_diversity_count_per_cbg.csv')

In [51]:
poi_diversity

,Unnamed: 0,cbg,diversity,Number
0,0,360810942032,2.428274,18
1,1,360471134003,0.636514,3
2,2,360470533005,1.945910,12
3,3,360810061002,1.810161,25
4,4,360610063005,2.415348,35
...,...,...,...,...
5976,5976,360050070002,1.515708,11
5977,5977,360610159002,1.549826,7
5978,5978,360470329003,2.484907,12
5979,5979,360050222001,1.979562,21


In [52]:
poi_with_stores = pd.merge(poi_diversity, geo_area_stores_gdf[['cbg', 'safegraph_place_id']], on = 'cbg')

In [53]:
store_count_dict = {}
store_diversity_dict = {}
for i,cont in poi_with_stores.iterrows():
    store_count_dict[cont.safegraph_place_id] = cont.Number
    store_diversity_dict[cont.safegraph_place_id] = cont.diversity

In [54]:
answers_df['POI_count_where_store_is'] = answers_df['safegraph_place_id'].apply(lambda x: store_count_dict[x])
answers_df['POI_diversity_where_store_is'] = answers_df['safegraph_place_id'].apply(lambda x: store_diversity_dict[x])

In [55]:
answers_df

,cbg,safegraph_place_id,Number_of_Visits_2020,Population_in_this_cbg,Median_Income_in_this_cbg,Population_where_store_is,Median_Income_where_store_is,Area_of_store,Distance_between_cbg_and_store,Includes_parking_lot,POI_count_where_store_is,POI_diversity_where_store_is
0,360810140001,sg:94b69491e8394408a41b4eaf79700a9a,0,2187,69750.0,1682,54861.0,12179,8.962305,0.0001,9,2.043192
1,360810140001,sg:dccb7bd06b53430e9aa215f4e47623f8,0,2187,69750.0,2453,41298.0,3715,8.322993,1.0000,16,1.927392
2,360810140001,sg:40c47fece0ba4897b01c7e28efb2731a,0,2187,69750.0,1407,76288.0,472341,3.419066,0.0001,33,2.834985
3,360810140001,sg:f51c84ff03e942c79e0fbf6e77caf930,0,2187,69750.0,1372,11818.0,4867,9.443017,0.0001,13,1.991509
4,360810140001,sg:7bf321e231b74c2f8e5a83727daca049,0,2187,69750.0,1639,50809.0,137041,11.710228,0.0001,20,2.510529
...,...,...,...,...,...,...,...,...,...,...,...,...
1831021,360470286004,sg:4217cdcdfa1a4bf7995658ea27daa03e,0,1147,59306.0,1058,40822.0,2370,6.062409,0.0001,25,2.414283
1831022,360470286004,sg:78948e77d30e426d90d4ae1998f336c0,0,1147,59306.0,142,0.0,133161,10.430985,0.0000,588,3.274630
1831023,360470286004,sg:eb3b350749604f60abc470b0ae0b4822,0,1147,59306.0,1491,16385.0,1626,18.916934,0.0000,47,2.702393
1831024,360470286004,sg:15682cf1eadb423297c1e634d13c0377,0,1147,59306.0,1283,152083.0,84616,11.337666,0.0001,103,2.620449


# M

In [66]:
census_similarity = pd.read_csv('data/1_data_for_extracting_information/NY_cbg_census.csv').drop(columns = ['Unnamed: 0','NAME'])
census_similarity['GEOID'] = census_similarity['GEOID'].astype(str)

In [67]:
census_similarity

,GEOID,populationE,med_ageE,bachelor_degree_25E,med_hh_incomeE,whiteE,blackE,asianE,hispanicE
0,360050245023,1886,28.9,80,30500.0,291,451,0,1648
1,360050247002,1318,35.4,106,44524.0,60,690,6,662
2,360050253004,1222,26.6,54,36250.0,115,45,0,1155
3,360050263005,1941,63.8,304,12351.0,604,428,0,891
4,360050265004,1191,28.2,87,38099.0,34,266,88,892
...,...,...,...,...,...,...,...,...,...
15458,360010129001,1328,28.4,127,53125.0,1067,18,0,14
15459,360010146082,1686,51.9,455,79457.0,1390,111,133,92
15460,360010146141,935,50.3,448,185435.0,859,0,76,0
15461,360010146144,1908,43.7,735,71932.0,1665,42,152,65


In [73]:
cbgs_s = [str(i) for i in cbgs]

In [74]:
census_similarity = census_similarity.rename(columns = {'GEOID': 'cbg'})
census_similarity = census_similarity[census_similarity['cbg'].isin(cbgs_s)]

In [75]:
census_similarity = census_similarity.fillna(0)
census_similarity

,cbg,populationE,med_ageE,bachelor_degree_25E,med_hh_incomeE,whiteE,blackE,asianE,hispanicE
0,360050245023,1886,28.9,80,30500.0,291,451,0,1648
1,360050247002,1318,35.4,106,44524.0,60,690,6,662
2,360050253004,1222,26.6,54,36250.0,115,45,0,1155
3,360050263005,1941,63.8,304,12351.0,604,428,0,891
4,360050265004,1191,28.2,87,38099.0,34,266,88,892
...,...,...,...,...,...,...,...,...,...
14060,360850207000,0,0.0,0,0.0,0,0,0,0
14061,360850208013,2110,51.9,373,54978.0,1969,0,141,96
14062,360850003000,0,0.0,0,0.0,0,0,0,0
14063,360850006002,1995,55.4,612,51964.0,1389,234,224,397


In [77]:
def get_pro(part):
    total_list = census_similarity.populationE.tolist()
    part_list = census_similarity[part].tolist()
    ans = []
    for i,num in enumerate(total_list):
        if num == 0:
            ans.append(0)
        else:
            ans.append(part_list[i]/num)
    return ans

In [78]:
census_similarity

,cbg,populationE,med_ageE,bachelor_degree_25E,med_hh_incomeE,whiteE,blackE,asianE,hispanicE
0,360050245023,1886,28.9,80,30500.0,291,451,0,1648
1,360050247002,1318,35.4,106,44524.0,60,690,6,662
2,360050253004,1222,26.6,54,36250.0,115,45,0,1155
3,360050263005,1941,63.8,304,12351.0,604,428,0,891
4,360050265004,1191,28.2,87,38099.0,34,266,88,892
...,...,...,...,...,...,...,...,...,...
14060,360850207000,0,0.0,0,0.0,0,0,0,0
14061,360850208013,2110,51.9,373,54978.0,1969,0,141,96
14062,360850003000,0,0.0,0,0.0,0,0,0,0
14063,360850006002,1995,55.4,612,51964.0,1389,234,224,397


In [79]:
change = ['bachelor_degree_25E', 'whiteE', 'blackE', 'asianE', 'hispanicE']
for i in change:
    census_similarity[i] = get_pro(i)

In [80]:
census_similarity

,cbg,populationE,med_ageE,bachelor_degree_25E,med_hh_incomeE,whiteE,blackE,asianE,hispanicE
0,360050245023,1886,28.9,0.042418,30500.0,0.154295,0.239130,0.000000,0.873807
1,360050247002,1318,35.4,0.080425,44524.0,0.045524,0.523520,0.004552,0.502276
2,360050253004,1222,26.6,0.044190,36250.0,0.094108,0.036825,0.000000,0.945172
3,360050263005,1941,63.8,0.156620,12351.0,0.311180,0.220505,0.000000,0.459042
4,360050265004,1191,28.2,0.073048,38099.0,0.028547,0.223342,0.073887,0.748950
...,...,...,...,...,...,...,...,...,...
14060,360850207000,0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
14061,360850208013,2110,51.9,0.176777,54978.0,0.933175,0.000000,0.066825,0.045498
14062,360850003000,0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
14063,360850006002,1995,55.4,0.306767,51964.0,0.696241,0.117293,0.112281,0.198997


In [81]:
census_similarity_compute = census_similarity.drop(columns = ['cbg', 'populationE'])
similarities = pd.DataFrame(cosine_similarity(census_similarity_compute, census_similarity_compute))

In [82]:
cbgs_in_queue = census_similarity.cbg.tolist()
def get_cos_sim(cbg_1, cbg_2):
    index_1 = cbgs_in_queue.index(str(cbg_1))
    index_2 = cbgs_in_queue.index(str(cbg_2))
    simi = similarities.iloc[index_1, index_2]
    if simi <= 1:
        return simi
    return 1

In [84]:
store_cbg_dict = {}
for i, cont in geo_area_stores_gdf.iterrows():
    store_cbg_dict[cont.safegraph_place_id] = cont.cbg

In [85]:
answers_df['Demographic_similarity'] = answers_df.apply(lambda row: get_cos_sim(row.cbg, store_cbg_dict[row.safegraph_place_id]), axis=1)

# P and Q

In [87]:
brand = table_2018[['B_store', 'O_Brand_name']].rename(columns = {'O_Brand_name': 'brands', 'B_store': 'safegraph_place_id'}).drop_duplicates()

In [88]:
brands_dict = {}
for i,cont in brand.iterrows():
    if cont.safegraph_place_id in final_stores:
        brands_dict[cont.safegraph_place_id] = cont.brands

In [89]:
answers_df['Brand_name'] = answers_df['safegraph_place_id'].apply(lambda x: brands_dict[x])

In [90]:
brands_id = pd.read_csv('data/1_data_for_extracting_information/brand_info.csv')

In [91]:
answers_df = pd.merge(answers_df, brands_id[['safegraph_brand_id', 'brand_name']], left_on = 'Brand_name', 
             right_on = 'brand_name').drop(columns = 'brand_name')

In [92]:
answers_df

,cbg,safegraph_place_id,Number_of_Visits_2020,Population_in_this_cbg,Median_Income_in_this_cbg,Population_where_store_is,Median_Income_where_store_is,Area_of_store,Distance_between_cbg_and_store,Includes_parking_lot,POI_count_where_store_is,POI_diversity_where_store_is,Demographic_similarity,Brand_name,safegraph_brand_id
0,360810140001,sg:94b69491e8394408a41b4eaf79700a9a,0,2187,69750.0,1682,54861.0,12179,8.962305,0.0001,9,2.043192,1.000000,Family Dollar Stores,SG_BRAND_6db086b96ff72a99a60c4a182d90fe10
1,360810140001,sg:b261ab19c88f4f68bb8efd01f0bea507,0,2187,69750.0,773,64643.0,8875,3.493805,0.0001,12,1.863680,1.000000,Family Dollar Stores,SG_BRAND_6db086b96ff72a99a60c4a182d90fe10
2,360810140001,sg:3f8d237767dc4dda8c8ed80f0e7d359d,0,2187,69750.0,676,0.0,8818,8.468017,0.0001,85,3.192731,0.999999,Family Dollar Stores,SG_BRAND_6db086b96ff72a99a60c4a182d90fe10
3,360810140001,sg:670f3d81cc5f44f5835954f0285f57af,0,2187,69750.0,1407,68259.0,4740,14.150168,0.0001,34,2.794355,1.000000,Family Dollar Stores,SG_BRAND_6db086b96ff72a99a60c4a182d90fe10
4,360810140001,sg:429f4ee166ff4523be060774d8354a8e,0,2187,69750.0,880,11268.0,9455,4.677972,0.0001,11,1.720193,0.999983,Family Dollar Stores,SG_BRAND_6db086b96ff72a99a60c4a182d90fe10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831021,360470271002,sg:e4019da3119b4ebfac7b8f9907d68d7c,0,980,50500.0,487,0.0,993,6.378445,1.0000,44,2.558892,0.000795,Bottega Veneta,SG_BRAND_8bdfb953dba680d2be0913fec2355417
1831022,360470280002,sg:e4019da3119b4ebfac7b8f9907d68d7c,8,865,58500.0,487,0.0,993,11.465520,1.0000,44,2.558892,0.000491,Bottega Veneta,SG_BRAND_8bdfb953dba680d2be0913fec2355417
1831023,360470282002,sg:e4019da3119b4ebfac7b8f9907d68d7c,0,1322,42019.0,487,0.0,993,11.518559,1.0000,44,2.558892,0.000567,Bottega Veneta,SG_BRAND_8bdfb953dba680d2be0913fec2355417
1831024,360470282003,sg:e4019da3119b4ebfac7b8f9907d68d7c,4,533,0.0,487,0.0,993,11.551485,1.0000,44,2.558892,0.999924,Bottega Veneta,SG_BRAND_8bdfb953dba680d2be0913fec2355417


# R and S

In [93]:
visits_by_brand = answers_df.groupby(['cbg', 'Brand_name'])['Number_of_Visits_'+str(year)].sum().reset_index().rename(
    columns = {'Number_of_Visits_'+str(year): 'Number_of_Visits_by_brand_'+str(year)})

In [94]:
counts_by_brand_df = visits_by_brand.groupby('cbg').sum().reset_index()
counts_by_brand_dict = {}
for i,cont in counts_by_brand_df.iterrows():
    counts_by_brand_dict[cont.cbg] = cont.Number_of_Visits_by_brand_2020

In [95]:
def compute_fraction_brand(row):
    total = counts_by_brand_dict[row.cbg]
    here = row['Number_of_Visits_by_brand_'+str(year)]
    if here == 0:
        return 0
    return here/total

In [96]:
visits_by_brand['Percentage_of_Visits_by_brand_'+str(year)] = visits_by_brand.apply(lambda row: compute_fraction_brand(row), axis=1)

In [97]:
answers_df = pd.merge(answers_df, visits_by_brand, on = ['cbg', 'Brand_name'])

In [98]:
answers_df

,cbg,safegraph_place_id,Number_of_Visits_2020,Population_in_this_cbg,Median_Income_in_this_cbg,Population_where_store_is,Median_Income_where_store_is,Area_of_store,Distance_between_cbg_and_store,Includes_parking_lot,POI_count_where_store_is,POI_diversity_where_store_is,Demographic_similarity,Brand_name,safegraph_brand_id,Number_of_Visits_by_brand_2020,Percentage_of_Visits_by_brand_2020
0,360810140001,sg:94b69491e8394408a41b4eaf79700a9a,0,2187,69750.0,1682,54861.0,12179,8.962305,0.0001,9,2.043192,1.000000,Family Dollar Stores,SG_BRAND_6db086b96ff72a99a60c4a182d90fe10,24,0.093023
1,360810140001,sg:b261ab19c88f4f68bb8efd01f0bea507,0,2187,69750.0,773,64643.0,8875,3.493805,0.0001,12,1.863680,1.000000,Family Dollar Stores,SG_BRAND_6db086b96ff72a99a60c4a182d90fe10,24,0.093023
2,360810140001,sg:3f8d237767dc4dda8c8ed80f0e7d359d,0,2187,69750.0,676,0.0,8818,8.468017,0.0001,85,3.192731,0.999999,Family Dollar Stores,SG_BRAND_6db086b96ff72a99a60c4a182d90fe10,24,0.093023
3,360810140001,sg:670f3d81cc5f44f5835954f0285f57af,0,2187,69750.0,1407,68259.0,4740,14.150168,0.0001,34,2.794355,1.000000,Family Dollar Stores,SG_BRAND_6db086b96ff72a99a60c4a182d90fe10,24,0.093023
4,360810140001,sg:429f4ee166ff4523be060774d8354a8e,0,2187,69750.0,880,11268.0,9455,4.677972,0.0001,11,1.720193,0.999983,Family Dollar Stores,SG_BRAND_6db086b96ff72a99a60c4a182d90fe10,24,0.093023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831021,360470271002,sg:e4019da3119b4ebfac7b8f9907d68d7c,0,980,50500.0,487,0.0,993,6.378445,1.0000,44,2.558892,0.000795,Bottega Veneta,SG_BRAND_8bdfb953dba680d2be0913fec2355417,0,0.000000
1831022,360470280002,sg:e4019da3119b4ebfac7b8f9907d68d7c,8,865,58500.0,487,0.0,993,11.465520,1.0000,44,2.558892,0.000491,Bottega Veneta,SG_BRAND_8bdfb953dba680d2be0913fec2355417,8,0.037383
1831023,360470282002,sg:e4019da3119b4ebfac7b8f9907d68d7c,0,1322,42019.0,487,0.0,993,11.518559,1.0000,44,2.558892,0.000567,Bottega Veneta,SG_BRAND_8bdfb953dba680d2be0913fec2355417,0,0.000000
1831024,360470282003,sg:e4019da3119b4ebfac7b8f9907d68d7c,4,533,0.0,487,0.0,993,11.551485,1.0000,44,2.558892,0.999924,Bottega Veneta,SG_BRAND_8bdfb953dba680d2be0913fec2355417,4,0.019900


# Percentage

In [99]:
answers_df_more = answers_df.copy()

In [100]:
answers_df_more

,cbg,safegraph_place_id,Number_of_Visits_2020,Population_in_this_cbg,Median_Income_in_this_cbg,Population_where_store_is,Median_Income_where_store_is,Area_of_store,Distance_between_cbg_and_store,Includes_parking_lot,POI_count_where_store_is,POI_diversity_where_store_is,Demographic_similarity,Brand_name,safegraph_brand_id,Number_of_Visits_by_brand_2020,Percentage_of_Visits_by_brand_2020
0,360810140001,sg:94b69491e8394408a41b4eaf79700a9a,0,2187,69750.0,1682,54861.0,12179,8.962305,0.0001,9,2.043192,1.000000,Family Dollar Stores,SG_BRAND_6db086b96ff72a99a60c4a182d90fe10,24,0.093023
1,360810140001,sg:b261ab19c88f4f68bb8efd01f0bea507,0,2187,69750.0,773,64643.0,8875,3.493805,0.0001,12,1.863680,1.000000,Family Dollar Stores,SG_BRAND_6db086b96ff72a99a60c4a182d90fe10,24,0.093023
2,360810140001,sg:3f8d237767dc4dda8c8ed80f0e7d359d,0,2187,69750.0,676,0.0,8818,8.468017,0.0001,85,3.192731,0.999999,Family Dollar Stores,SG_BRAND_6db086b96ff72a99a60c4a182d90fe10,24,0.093023
3,360810140001,sg:670f3d81cc5f44f5835954f0285f57af,0,2187,69750.0,1407,68259.0,4740,14.150168,0.0001,34,2.794355,1.000000,Family Dollar Stores,SG_BRAND_6db086b96ff72a99a60c4a182d90fe10,24,0.093023
4,360810140001,sg:429f4ee166ff4523be060774d8354a8e,0,2187,69750.0,880,11268.0,9455,4.677972,0.0001,11,1.720193,0.999983,Family Dollar Stores,SG_BRAND_6db086b96ff72a99a60c4a182d90fe10,24,0.093023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831021,360470271002,sg:e4019da3119b4ebfac7b8f9907d68d7c,0,980,50500.0,487,0.0,993,6.378445,1.0000,44,2.558892,0.000795,Bottega Veneta,SG_BRAND_8bdfb953dba680d2be0913fec2355417,0,0.000000
1831022,360470280002,sg:e4019da3119b4ebfac7b8f9907d68d7c,8,865,58500.0,487,0.0,993,11.465520,1.0000,44,2.558892,0.000491,Bottega Veneta,SG_BRAND_8bdfb953dba680d2be0913fec2355417,8,0.037383
1831023,360470282002,sg:e4019da3119b4ebfac7b8f9907d68d7c,0,1322,42019.0,487,0.0,993,11.518559,1.0000,44,2.558892,0.000567,Bottega Veneta,SG_BRAND_8bdfb953dba680d2be0913fec2355417,0,0.000000
1831024,360470282003,sg:e4019da3119b4ebfac7b8f9907d68d7c,4,533,0.0,487,0.0,993,11.551485,1.0000,44,2.558892,0.999924,Bottega Veneta,SG_BRAND_8bdfb953dba680d2be0913fec2355417,4,0.019900


In [101]:
def compute_fraction(row, y):
    total = counts_by_brand_dict[row.cbg]
    here = row['Number_of_Visits_'+str(y)]
    if here == 0:
        return 0
    return here/total

In [102]:
def compute_fraction_brand(row, y):
    total = counts_by_brand_dict[row.cbg]
    here = row['Number_of_Visits_by_brand_'+str(y)]
    if here == 0:
        return 0
    return here/total

In [103]:
answers_df_more['Percentage_of_Visits_'+str(year)] = answers_df_more.apply(lambda row: compute_fraction(row, year), axis = 1)
answers_df_more['Percentage_of_Visits_by_brand_'+str(year)] = answers_df_more.apply(lambda row: compute_fraction_brand(row, year), axis = 1)

In [104]:
table_2018.shape

(1831026, 19)

In [105]:
def get_table(year):
    temp = answers_df_more.copy()
    
    change_dict = {'cbg': 'A_cbg', 'safegraph_place_id': 'B_store', 
                   'Percentage_of_Visits_'+str(year): 'C_Percentage_of_Visits_'+str(year), 
                  'Number_of_Visits_'+str(year): 'D_Number_of_Visits_'+str(year), 
                   'Population_in_this_cbg': 'E_Population_in_this_cbg', 
                  'Population_where_store_is': 'F_Population_where_store_is', 
                   'Distance_between_cbg_and_store': 'G_Distance_between_cbg_and_store', 
                   'Area_of_store': 'H_Area_of_store',
                  'Includes_parking_lot': 'I_Includes_parking_lot', 
                   'POI_count_where_store_is': 'J_POI_count_where_store_is',
                  'POI_diversity_where_store_is': 'K_POI_diversity_where_store_is', 
                   'Demographic_similarity': 'L_Demographic_similarity',
                  'Median_Income_in_this_cbg': 'M_Median_Income_in_this_cbg', 
                   'Median_Income_where_store_is': 'N_Median_Income_where_store_is', 'Brand_name': 'O_Brand_name',
                  'safegraph_brand_id': 'P_safegraph_brand_id', 
                   'Number_of_Visits_by_brand_' + str(year): 'Q_Number_of_Visits_by_brand_' + str(year),
                  'Percentage_of_Visits_by_brand_' + str(year): 'R_Percentage_of_Visits_by_brand_' + str(year)}
    
    temp = temp.rename(columns = change_dict).sort_index(axis=1)
    return temp

In [106]:
table_2020 = get_table(2020)

In [107]:
table_2020.columns

Index(['A_cbg', 'B_store', 'C_Percentage_of_Visits_2020',
       'D_Number_of_Visits_2020', 'E_Population_in_this_cbg',
       'F_Population_where_store_is', 'G_Distance_between_cbg_and_store',
       'H_Area_of_store', 'I_Includes_parking_lot',
       'J_POI_count_where_store_is', 'K_POI_diversity_where_store_is',
       'L_Demographic_similarity', 'M_Median_Income_in_this_cbg',
       'N_Median_Income_where_store_is', 'O_Brand_name',
       'P_safegraph_brand_id', 'Q_Number_of_Visits_by_brand_2020',
       'R_Percentage_of_Visits_by_brand_2020'],
      dtype='object')

In [108]:
table_2020.to_csv('data/2_data_for_PSO_calibration/table_2020.csv', index = False)